#### 论文中提到FCN可以输入任意尺寸的图片，然后输出的mask同原图大小一样，但是在实际训练的时候，为了进行一个batch的训练，需要将图片大小设置为一样，pascal voc 2012的图片大小是不一样的，这儿统一通过pad和crop成（320，320）的大小

In [1]:
import os

In [2]:
train_images_list=os.listdir('/input0/pascal_2012_seg/train/images')

In [3]:
print(len(train_images_list))

2330


#### 创建路径

In [ ]:
os.mkdir('./data')
os.mkdir('./data/train')
os.mkdir('./data/validation')
os.mkdir('./data/test')
os.mkdir('./data/train/images')
os.mkdir('./data/train/masks')
os.mkdir('./data/validation/images')
os.mkdir('./data/validation/masks')
os.mkdir('./data/test/images')
os.mkdir('./data/test/masks')


#### 先对图片进行填充再进行裁剪，图片长宽小于320的则填充0，所以填充后的图片的长宽都是大于等于320的，然后将图片尺寸大于320的进行随机裁剪，裁剪的时候必须得保证images和masks的裁剪是在对应的区域进行裁剪，这样才能保证裁剪后images和masks的像素依然是一一对应的

In [6]:
from PIL import Image
from keras.preprocessing import image
import numpy as np
base_path='/input0/pascal_2012_seg/'
# val_base_path='/input0/pascal_2012_seg/validation'

def pair_random_crop(name,data_class,images_save_path,masks_save_path,target_size ,sync_seed=1, **kwargs):
    np.random.seed(sync_seed)
    # 填充
    x_ori_img=image.load_img(base_path+data_class+'/images/'+name+'.jpg')
    x=image.img_to_array(x_ori_img)
    
    y_ori_img=image.load_img(base_path+data_class+'/masks/'+name+'.png')
    y=image.img_to_array(y_ori_img)

    h, w = x.shape[0], x.shape[1]
#     print(h,w)
#     print(target_size[1])
    pad_w = max(target_size[1] - w, 0)
    pad_h = max(target_size[0] - h, 0)
    
    x = np.lib.pad(x, ((pad_h // 2, pad_h - pad_h // 2), (pad_w // 2, pad_w - pad_w // 2), (0, 0)),
                   'constant', constant_values=0.)
    y = np.lib.pad(y, ((pad_h // 2, pad_h - pad_h // 2), (pad_w // 2, pad_w - pad_w // 2), (0, 0)),
                   'constant', constant_values=0.)

#     print(x.shape,y.shape)
    # h, w = x.shape[0], x.shape[1]
    #裁减
    h, w = x.shape[0], x.shape[1]
    rangeh = (h - target_size[0]) // 2
    rangew = (w - target_size[1]) // 2
#     print(rangeh,rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)

    h_start, h_end = offseth, offseth + target_size[0]
    w_start, w_end = offsetw, offsetw + target_size[1]
    x_img=x[h_start:h_end, w_start:w_end, :]
    im=Image.fromarray(x_img.astype('uint8'))
    im.save(images_save_path+'/'+name+'.jpg')
    
    y_mask=y[h_start:h_end, w_start:w_end, :]
    im=Image.fromarray(y_mask.astype('uint8'))
    im.save(masks_save_path+'/'+name+'.png')

    return x[h_start:h_end, w_start:w_end, :], y[h_start:h_end, w_start:w_end, :]

In [ ]:
# train_images_list
train_images_list=os.listdir('/input0/pascal_2012_seg/train/images')
for each in train_images_list:
#     print(each)
        each=each.split('.')
        name=each[0]
#         print(name)
        pair_random_crop(name,'train','./data/train/images','./data/train/masks',target_size=(320,320))
        
print('Done')  

In [12]:
# train_images_list
val_list=os.listdir('/input0/pascal_2012_seg/validation/images')
for each in val_list:
#     print(each)
        each=each.split('.')
        name=each[0]
#         print(name)
        pair_random_crop(name,'validation','./data/validation/images','./data/validation/masks',(320,320))
        
print('Done')  

Done


In [13]:
# test_images_list
val_list=os.listdir('/input0/pascal_2012_seg/test/images')
for each in val_list:
#     print(each)
        each=each.split('.')
        name=each[0]
#         print(name)
        pair_random_crop(name,'test','./data/test/images','./data/test/masks',(320,320))
        
print('Done')  

Done
